### --- Day 8: Haunted Wasteland ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2023/day/8

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2023, 8);
WriteLines(inputLines);

Loading puzzle file: Day8.txt
Total lines: 764
Max line length: 263

LRRRLRLLLLLLLRLRLRRLRRRLRRLRRRLRRLRRRLLRRRLRRLRLRRRLRRLRRRLLRLLRRRLRRRLRLLRLRLRRRLRRLRRLRRLRLRRRLRRLRRRLLRLLRLLRRLRLLRLRRLRLRLRRLRRRLLLRRLRRRLLRRLRLRLRRRLRLRRRLLRLLLRRRLLLRRLLRLLRRLLRLRRRLRLRRLRRLLRRLRLLRLRRRLRRRLRLRRRLRLRLRRLRLRRRLRRRLRRRLRRLRRLRRRLLRLRLLRLLRRRR

HVX = (SCS, XQN)
DMK = (JKL, JKL)
FDF = (XHL, RMM)


In [4]:
var instructions = inputLines[0];

In [5]:
record struct Node(string Key, string LeftKey, string RightKey);

In [6]:
Node ParseNode(string line) {
    var key = line.Substring(0, 3);
    var leftKey = line.Substring(7, 3);
    var rightKey = line.Substring(12, 3);

    var result = new Node(key, leftKey, rightKey);
    return result;
}

var testLine = "HVX = (SCS, XQN)";
Console.WriteLine(ParseNode(testLine));

Node { Key = HVX, LeftKey = SCS, RightKey = XQN }


In [7]:
var allParsedNodes = inputLines[2..].Select(ParseNode);

var network = allParsedNodes.ToDictionary(n => n.Key);

foreach (var n in network.Take(10)) {
    Console.WriteLine(n);
}

[HVX, Node { Key = HVX, LeftKey = SCS, RightKey = XQN }]
[DMK, Node { Key = DMK, LeftKey = JKL, RightKey = JKL }]
[FDF, Node { Key = FDF, LeftKey = XHL, RightKey = RMM }]
[JTK, Node { Key = JTK, LeftKey = SVN, RightKey = DVP }]
[QCF, Node { Key = QCF, LeftKey = FCH, RightKey = FCH }]
[TCG, Node { Key = TCG, LeftKey = VMS, RightKey = SDL }]
[JJP, Node { Key = JJP, LeftKey = FQJ, RightKey = RLT }]
[DRP, Node { Key = DRP, LeftKey = RMJ, RightKey = RMJ }]
[VKF, Node { Key = VKF, LeftKey = XQB, RightKey = VBX }]
[HRS, Node { Key = HRS, LeftKey = BXK, RightKey = DPM }]


In [8]:
IEnumerable<char> RepeatSteps(string steps) {
    while (true) {
        foreach (var step in steps) {
            yield return step;
        }
    }
}

In [9]:

var currentStep = network["AAA"];
var steps = 0;

foreach (var step in RepeatSteps(instructions)) {
    var nextKey = step switch {
        'L' => currentStep.LeftKey,
        _ => currentStep.RightKey
    };

    currentStep = network[nextKey];
    steps++;

    if (currentStep.Key == "ZZZ") {
        break;
    }

    if (steps >= 100000) {
        throw new Exception("Too many steps");
    }
}

// Starting at AAA, follow the left/right instructions. How many steps are required to reach ZZZ?

var part1Answer = steps;
Console.WriteLine(part1Answer);

15517


In [10]:
// 15517 is correct!
Ensure(15517, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2023/day/8

In [12]:
(int initial, int loop) FindLoop(string startPos) 
{
    var currentStep = network[startPos];
    var steps = 0;

    int initial = 0;
    int loop = 0;
    foreach (var step in RepeatSteps(instructions)) {
        var nextKey = step == 'L' ? currentStep.LeftKey : currentStep.RightKey;

        currentStep = network[nextKey];
        steps++;

        if (currentStep.Key.EndsWith('Z')) {
            if (initial == 0) {
                initial = steps;
            } else {
                loop = (steps - initial);
                break;
            }
        }

        if (steps >= 100000) {
            throw new Exception("Too many steps");
        }
    }

    return (initial, loop);
}

In [13]:
var startKeys = network.Keys.Where(k => k.EndsWith('A')).ToArray();
var loops = startKeys.Select(FindLoop).ToArray();

foreach (var l in loops) {
    Console.WriteLine(l);
}

// Interestingly, they all reach their initial point and loop at the same interval! Nice, we can use that

var loopPeriods = loops.Select(l => l.loop).ToArray();

static ulong GCD(ulong a, ulong b) {
    if (b == 0) {
        return a;
    }

    return GCD(b, a % b);
}

// Least common multiple. I.e., If we repeat a and b, at what value do they equal?
static ulong LCM(ulong a, ulong b) {
    var gcd = GCD(a, b);
    var result = a * (b / gcd);

    return result;
}

(13939, 13939)
(17621, 17621)
(19199, 19199)
(15517, 15517)
(12361, 12361)
(20777, 20777)


In [14]:
// LCM is ideal for us as we are basically want to know the value at which 2
// cycles looping at different frequencies will both reach the origin

// In our case, the LCM of the first 2 loops gives us a new "minimum" cycle
// time. Therefore we feed this cycle into the next loop input, which gives us the
// next cycle, and so on

var part2Answer = loopPeriods.Select(l => (ulong)l).Aggregate(LCM);

foreach (var l in loopPeriods) {
    var x = part2Answer % (ulong)l;
    Console.WriteLine($"{part2Answer} % {l} = {x}");
}

Console.WriteLine(part2Answer);


14935034899483 % 13939 = 0
14935034899483 % 17621 = 0
14935034899483 % 19199 = 0
14935034899483 % 15517 = 0
14935034899483 % 12361 = 0
14935034899483 % 20777 = 0
14935034899483


In [15]:
// 14935034899483 is correct!
Ensure(14935034899483, part2Answer);